# STEP 0 Check GPU  

## This notebook must be run in GPU mode. You can run the nvidia-smi command above to check if the GPU is loaded. If not, click on the top-left corner > Edit > Notebook settings, and change the hardware accelerator to GPU.  






In [1]:
!nvidia-smi

Wed Oct 18 06:38:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title STEP 1 Clone repository
!git clone https://github.com/LIYUESEN/druggpt.git
!pwd
import os
os.chdir("/content/druggpt")
!pwd

Cloning into 'druggpt'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 95 (delta 26), reused 5 (delta 4), pack-reused 56
Receiving objects: 100% (95/95), 50.74 KiB | 742.00 KiB/s, done.
Resolving deltas: 100% (44/44), done.
/content
/content/druggpt


In [3]:
#@title STEP 2 Build environment
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda config --set always_yes yes --set changeps1 no
!conda update -q conda

!conda create -y -n druggpt python=3.7
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

!source activate druggpt && pip install --root-user-action=ignore datasets transformers scipy scikit-learn
!source activate druggpt && pip install --root-user-action=ignore torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!source activate druggpt && conda install -c openbabel openbabel -y
!source activate druggpt && conda list

--2023-10-18 06:38:42--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103219356 (98M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  98.44M   307MB/s    in 0.3s    

2023-10-18 06:38:43 (307 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [103219356/103219356]

PREFIX=/usr/local
Unpacking payload ...
                                                                                 
Installing base environment...





Preparing transaction: - \ | / done
Executing transaction: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
installation finished.
    You currently have a P

In [5]:
#@title STEP 3 Run druggpt

#@markdown # **Instructions**
#@markdown - If you select "protein amino acid sequence", input a protein amino acid sequence in the input_protein_amino_acid_sequence field.
#@markdown - If you select "fasta file", first upload the fasta file to the druggpt directory by dragging and dropping it into the file browser on the left side. Then, input the name of the fasta file as text in the input_fasta_file field.
#@markdown - If "no input" is selected, the input content will be ignored.

#@markdown #step 3.1 Select your input type
Input_type = "fasta file" #@param ["protein amino acid sequence", "fasta file", "no input"]
#@markdown #step 3.2 Provide the required input based on your selected input type:

#@markdown - For "protein amino acid sequence", input a protein amino acid sequence.
input_protein_amino_acid_sequence = "MAKQPSDVSSECDREGRQLQPAERPPQLRPGAPTSLQTEPQGNPEGNHGGEGDSCPHGSPQGPLAPPASPGPFATRSPLFIFMRRSSLLSRSSSGYFSFDTDRSPAPMSCDKSTQTPSPPCQAFNHYLSAMASMRQAEPADMRPEIWIAQELRRIGDEFNAYYARRVFLNNYQAAEDHPRMVILRLLRYIVRLVWRMH" #@param {type:"raw"}
#@markdown - Upload the fasta file to the "druggpt" directory by selecting the file and uploading it. Ensure that the file is placed within the same folder as the default "bcl2.fasta" file. Then, input the name of the fasta file in the text field.
input_fasta_flie = "bcl2.fasta" #@param {type:"raw"}
if Input_type == "protein amino acid sequence" :
  input_content = input_protein_amino_acid_sequence
elif Input_type == "fasta file":
  input_content = input_fasta_flie
else:pass




#@markdown #step 3.3 Choose whether to use ligand prompt
ligand_prompt = False #@param {type:"boolean"}
#@markdown -  If ligand prompt is not checked, the ligand prompt content will be ignored.
if ligand_prompt == True:
  ligand_prompt_content = "COc1ccc(cc1)C(=O)" #@param {type:"raw"}
  ligand_content = "\'" + ligand_prompt_content + "\'"


#@markdown #step 3.4 Set Additional Parameters

#@markdown - At least how many molecules will be generated.
number = 50 #@param {type:"integer"}

#@markdown - Hardware device to use. Default is 'cuda'.
device = "cuda" #@param ["cuda", "cpu"]

#@markdown - Output directory for generated molecules. Default is './ligand_output/'.
output = './ligand_output/' #@param {type:"raw"}

#@markdown - how many molecules will be generated per batch. Try to reduce this value if you have low RAM. Default is 32.
batch_size = 32 #@param {type:"integer"}

#@markdown # Don't forget to run the cell.
temp = "source activate druggpt && TOKENIZERS_PARALLELISM=false && python drug_generator.py "
if Input_type == "no input":
  temp = temp + "-e "
elif Input_type == "protein amino acid sequence":
  temp = temp + "-p " + input_content + " "
else:
  temp = temp + "-f " + input_content + " "

if (ligand_prompt == True) and (Input_type != "no input"):
  temp = temp + "-l " + ligand_content + " "

temp = temp + "-n " + str(number) + " -d " + device + " -o " +output + " -b "+ str(batch_size)
print(temp)

!{temp}



source activate druggpt && TOKENIZERS_PARALLELISM=false && python drug_generator.py -f bcl2.fasta -n 50 -d cuda -o ./ligand_output/ -b 32

  _____                    _____ _____ _______ 
 |  __ \                  / ____|  __ \__   __|
 | |  | |_ __ _   _  __ _| |  __| |__) | | |   
 | |  | | '__| | | |/ _` | | |_ |  ___/  | |   
 | |__| | |  | |_| | (_| | |__| | |      | |   
 |_____/|_|   \__,_|\__, |\_____|_|      |_|   
                     __/ |                     
                    |___/                      
 A generative drug design model based on GPT2
    
<|startoftext|><P>MAKQPSDVSSECDREGRQLQPAERPPQLRPGAPTSLQTEPQGNPEGNHGGEGDSCPHGSPQGPLAPPASPGPFATRSPLFIFMRRSSLLSRSSSGYFSFDTDRSPAPMSCDKSTQTPSPPCQAFNHYLSAMASMRQAEPADMRPEIWIAQELRRIGDEFNAYYARRVFLNNYQAAEDHPRMVILRLLRYIVRLVWRMH<L>
=====Batch 1=====
Generating ligand SMILES ...
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtai

In [6]:
#@title STEP 4 Save results
#@markdown # Enter the name of the results folder you want to download:
from google.colab import files
results_dir_name = 'ligand_output' #@param {type:"raw"}
results_zip = results_dir_name + ".zip"
print(results_zip)
!zip -r $results_zip $results_dir_name
files.download(results_zip)

ligand_output.zip
  adding: ligand_output/ (stored 0%)
  adding: ligand_output/05a5c85b6c286fefe528d89bdf39636beb381ef0.sdf (deflated 80%)
  adding: ligand_output/6d175dbe3daa7e628e4cad13f731c9bc5b32498b.sdf (deflated 80%)
  adding: ligand_output/2b1aaa69f3043a4ca808c36dace8aee2c8da6c9e.sdf (deflated 79%)
  adding: ligand_output/6db8fe63abc6db0a07f81fd6c0ba056639758dfe.sdf (deflated 80%)
  adding: ligand_output/b42ddfe63b686b66a0ce8cbb1e717f21b09e3d7d.sdf (deflated 80%)
  adding: ligand_output/748814ab31e1eaf13042ed9a579871f50dc53847.sdf (deflated 79%)
  adding: ligand_output/933fba1d9797d71aa1db05139a36c7745b385b20.sdf (deflated 80%)
  adding: ligand_output/b27c7e91ea1a3e392c0b5bc553346eeadd70560e.sdf (deflated 80%)
  adding: ligand_output/1dab9f4e2150ebe7d42aaa2eca6b518236c4b57d.sdf (deflated 79%)
  adding: ligand_output/90b94096f5e8c08c234df400dd54995aa313ce6e.sdf (deflated 78%)
  adding: ligand_output/f6c3410ef84dffe0f67d0361340173f96936976b.sdf (deflated 79%)
  adding: ligand_outp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Step 5: Visualize and post-process on your local computer  

## After completing the model predictions and downloading the results, you can visualize and post-process the results on your local computer. Here are some suggested steps:

## 1. Unzip the downloaded results folder: Locate the zip file you downloaded in Step 4 and unzip it to access the result files.

## 2. Install molecular visualization software: Before visualizing, make sure you have installed molecular visualization software, such as PyMOL, UCSF Chimera, or Discovery Studio Visualizer. These tools can help you visualize the protein structures and ligands more intuitively.

## 3. Open the protein and ligand structures: Use the molecular visualization software to open the protein structure file (e.g., in PDB format) and the ligand structure files from the downloaded results (e.g., in SDF format).

## 4. Analyze the predicted results: When visualizing the protein-ligand complexes, pay attention to the following aspects:

- The binding position and pose of the ligand with the protein  
- Hydrogen bonds, hydrophobic interactions, and other interactions between the ligand and protein  
- Any conformational changes that might affect binding  
- The consistency of the results with known experimental data (e.g., crystal structures, binding assays, etc.)

## 5. Further post-processing: As needed, you can use other bioinformatics tools (e.g., AutoDock, GROMACS, or Amber) to perform energy minimization, molecular dynamics simulations, or other analyses on the predicted results.

## 6. Share the results: If you are satisfied with the analysis results, you can share your findings with colleagues, collaborators, or the community. This can be done by writing reports, creating presentations, or publishing online resources.